In [5]:
import keras
import sys
sys.path.append('kernel')
import DigitRecognizer; reload(DigitRecognizer)
import numpy as np

In [2]:
datapath='../data/mnist/'

### Preprocessing

In [3]:
#Take training.csv and create a validation.csv
#this loses 2 training examples since I'm guessing due to non inclusivity of ranges
import random
f = open(datapath + 'train.csv', "r")
v = open(datapath + 'validation.csv', "w")
t = open(datapath + 'training.csv', "w")
f.readline()
lines = []
for l in f:
    lines.append(l)

random.shuffle(lines)
for l in lines[:len(lines)/10]:
    v.write(l)
for l in lines[len(lines)/10:]:
    t.write(l)


In [4]:
tr = open(datapath+'training.csv', "r")
line = tr.readline()
values = line.split(',')
clss = values[0]
pixels = values[1:]
print(len(pixels), len(values))
output = np.zeros((28,28))
for i in range(0,28):
    for j in range(0,28):
        output[i,j] = pixels[(i * 28) + j]


(784, 785)


ValueError: could not convert string to float: pixel0

In [11]:
print keras.__version__

2.0.4


### Get Batches


In [ ]:
#Get model and get batches as numpy array (28x28) pixels and labels (0..9)
model = DigitRecognizer.DigitRecognizer()
training_data,training_y = model.get_training_data(datapath + 'train_no_header.csv')
training_labels = keras.utils.to_categorical(training_y, num_classes=10)
#validation_batches = model.get_training_data(datapath + 'validation.csv')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 7, 26, 26)         70        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 24, 24)         448       
_________________________________________________________________
flatten_2 (Flatten)          (None, 4032)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 4024)              16228792  
_________________________________________________________________
dense_4 (Dense)              (None, 10)                40250     
Total params: 16,269,560
Trainable params: 16,269,560
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#print(training_labels)
#print(training_y)
np.set_printoptions(threshold='nan')
print(training_data)
np.set_printoptions(threshold=1000)

### Train the model

In [ ]:
#model.fit(training_data, validation_data, nb_epoch=1)
model.fit(training_data, training_labels, nb_epoch=10,validation_split=.1)

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 [==============================] - 11s - loss: 12.2423 - acc: 0.2397 - val_loss: 9.6687 - val_acc: 0.3998

### Predict results

In [60]:
test_batches = model.get_test_data(datapath + 'test.csv')
predictions = model.predict(test_batches)

In [62]:
print(predictions.shape, test_batches.shape)
print(predictions)
#np.savetxt("training_data.txt", training_data[])
np.savetxt("training_labels.txt", training_labels)
np.savetxt("predictions.txt", predictions)

((28001L, 10L), (28001L, 1L, 28L, 28L))
[[  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   7.25705984e-09   6.17795282e-09 ...,   5.69956615e-09
    7.65915242e-09   3.54335805e-09]
 [  1.00000000e+00   7.25705984e-09   6.17795282e-09 ...,   5.69956615e-09
    7.65915242e-09   3.54335805e-09]
 ..., 
 [  1.00000000e+00   7.25705984e-09   6.17795282e-09 ...,   5.69956615e-09
    7.65915242e-09   3.54335805e-09]
 [  1.00000000e+00   7.25705984e-09   6.17795282e-09 ...,   5.69956615e-09
    7.65915242e-09   3.54335805e-09]
 [  1.00000000e+00   7.25700433e-09   6.17788221e-09 ...,   5.69947867e-09
    7.65910801e-09   3.54331742e-09]]


In [51]:
out = open("submission.csv", "w")
out.write("ImageId,Label\n")


In [52]:
for i in range(0,predictions.shape[0]):
    for j in range(0,predictions.shape[1]):
        if (predictions[i,j] > .5):
            out.write(str(i+1) + "," + str(j) + "\n")